In [2]:
# This notebook is an example of Pandas chaining using lambda functions that I made to tweak 
# a blockchain csv output for tax purposes.  This style of coding in pandas avoids the setting with 
# copy warning, cuts down on memory usage by not assigning intermediate variables and is easily 
# reproducable with one funtion call.  

In [3]:
import datetime
import pandas as pd
import os
import numpy as np

In [4]:
path = os.path.join(os.getcwd(), "blockchain_explorer_outputs")
immutable = pd.read_csv(os.path.join(path, 'gamestop_wallet_immutable.csv'))
immutable.txn_type.value_counts()

transfer    130
buy          81
mint         77
deposit       9
sell          1
Name: txn_type, dtype: int64

In [5]:
immutable.columns

Index(['txn_id', 'txn_time', 'txn_type', 'from_address', 'to_address',
       'token_address', 'collection_name', 'collection_image_uri', 'token_id',
       'token_name', 'token_image_uri', 'token_symbol', 'token_quantity',
       'usd_amount'],
      dtype='object')

In [6]:
immutable.head()

,txn_id,txn_time,txn_type,from_address,to_address,token_address,collection_name,collection_image_uri,token_id,token_name,token_image_uri,token_symbol,token_quantity,usd_amount
0,124991094,2022-10-08T23:02:15.946Z,mint,NaN,NaN,0xacb3c6a43d15b907e8433077b6d38ae40936fe2c,Gods Unchained Cards,https://images.godsunchained.com/misc/gu-sigel...,211056767.0,Lootable Corpse,https://card.godsunchained.com/?id=1681&q=3,NaN,NaN,NaN
1,124991095,2022-10-08T23:02:15.946Z,mint,NaN,NaN,0xacb3c6a43d15b907e8433077b6d38ae40936fe2c,Gods Unchained Cards,https://images.godsunchained.com/misc/gu-sigel...,211056768.0,On Her Command,https://card.godsunchained.com/?id=1609&q=4,NaN,NaN,NaN
2,124991096,2022-10-08T23:02:15.946Z,mint,NaN,NaN,0xacb3c6a43d15b907e8433077b6d38ae40936fe2c,Gods Unchained Cards,https://images.godsunchained.com/misc/gu-sigel...,211056769.0,Surpassing Blast,https://card.godsunchained.com/?id=1527&q=4,NaN,NaN,NaN
3,124991097,2022-10-08T23:02:15.946Z,mint,NaN,NaN,0xacb3c6a43d15b907e8433077b6d38ae40936fe2c,Gods Unchained Cards,https://images.godsunchained.com/misc/gu-sigel...,211056770.0,Engaged Healer,https://card.godsunchained.com/?id=1504&q=4,NaN,NaN,NaN
4,124991098,2022-10-08T23:02:15.946Z,mint,NaN,NaN,0xacb3c6a43d15b907e8433077b6d38ae40936fe2c,Gods Unchained Cards,https://images.godsunchained.com/misc/gu-sigel...,211056771.0,Ash Reader,https://card.godsunchained.com/?id=1535&q=4,NaN,NaN,NaN


In [47]:
def tweak_dataframe(df_ = immutable):
    """Converts immutable csv dataframe into a form acceptable by koinly tax software"""
    def combine_columns(df_, columns:list):
        """Function to combine calculated fields for specific transaction types"""
        series = df_[columns[0]]
        del columns[0]
        while len(columns) > 0:
            series = series.combine_first(other=df_[columns[0]])
            del columns[0]
        return series
    
    token_ids = immutable.token_id.dropna().drop_duplicates()
    id_map = dict(zip(token_ids, token_ids.index+1000))
    
    return (df_
    # formatting date column        
    .assign(
        Date=lambda df_: pd.to_datetime(df_.txn_time).dt.strftime('%Y-%m-%d %H:%M %Z'))
    # dealing with the buy transaction types
    .assign(
        Sent_Amount_Buy=lambda df_: df_.token_quantity.where(df_.txn_type=='buy', np.nan),
        Sent_Currency_Buy=lambda df_: df_.token_symbol.where(df_.txn_type=='buy', np.nan),
        Received_Amount_Buy=lambda df_: np.where(df_.txn_type=='buy', 1, np.nan),
        Received_Currency_Buy=lambda df_: df_.token_id.where(df_.txn_type=='buy', np.nan))
    # dealing with sell transaction types
    .assign(
        Received_Amount_Sell=lambda df_: df_.token_quantity.where(df_.txn_type=='sell', np.nan),
        Received_Currency_Sell=lambda df_: df_.token_symbol.where(df_.txn_type=='sell', np.nan),
        Sent_Amount_Sell=lambda df_: np.where(df_.txn_type=='sell', 1, np.nan),
        Sent_Currency_Sell=lambda df_: df_.token_id.where(df_.txn_type=='sell', np.nan))
    # dealing with mint transaction types
    .assign(
        Received_Amount_Mint=lambda df_: np.where(df_.txn_type=='mint', 1, np.nan),
        Received_Currency_Mint=lambda df_: df_.token_id.where(df_.txn_type=='mint', np.nan))
    # dealing with transfer transaction types
    #   First, dealing with transfers sent from the wallet
    .assign(
        Sent_Amount_Transfer=lambda df_: df_.token_quantity.where(
            (df_.txn_type=="transfer")&(~df_.to_address.isna())),
        Sent_Currency_Transfer=lambda df_: df_.token_symbol.where(
            (df_.txn_type=="transfer")&(~df_.to_address.isna())),
        Sent_Amount_Transfer_Nft=lambda df_: np.where(
            ((df_.txn_type=='transfer')&
             (~df_.to_address.isna())&
             (~df_.token_id.isna())),
            1, np.nan),
        Sent_Currency_Transfer_Nft=lambda df_: df_.token_id.where(
            ((df_.txn_type=='transfer')&
             (~df_.to_address.isna())&
             (~df_.token_id.isna()))))
    #   Next, dealing with transfers received by the wallet
    .assign(
        Received_Amount_Transfer=lambda df_: df_.token_quantity.where(
            (df_.txn_type=="transfer")&(~df_.from_address.isna())),
        Received_Currency_Transfer=lambda df_: df_.token_symbol.where(
            (df_.txn_type=="transfer")&(~df_.from_address.isna())),
        Received_Amount_Transfer_Nft=lambda df_: np.where(
            ((df_.txn_type=='transfer')&
             (~df_.from_address.isna())&
             (df_.token_id.isna())),
            1, np.nan),
        Received_Currency_Transfer_Nft=lambda df_: df_.token_id.where(
            ((df_.txn_type=='transfer')&
             (~df_.from_address.isna())&
             (~df_.token_id.isna()))))
    # dealing with deposit transaction types
    .assign(
        Received_Amount_Deposit=lambda df_:
            df_.token_quantity.where(df_.txn_type=="deposit",np.nan),
        Received_Currency_Deposit=lambda df_:
            df_.token_symbol.where(df_.txn_type=="deposit", np.nan))
    # Merging together applicable columns
    .assign(
        Sent_Amount=lambda df_: combine_columns(df_, columns=[
            'Sent_Amount_Buy', 'Sent_Amount_Transfer',
            'Sent_Amount_Transfer_Nft', 'Sent_Amount_Sell']),
        Sent_Currency=lambda df_: combine_columns(df_, columns=[
            'Sent_Currency_Buy', 'Sent_Currency_Transfer',
            'Sent_Currency_Transfer_Nft', 'Sent_Currency_Sell']),
        Received_Amount=lambda df_: combine_columns(df_, columns=[
            'Received_Amount_Buy', 'Received_Amount_Sell', 'Received_Amount_Transfer',
            'Received_Amount_Transfer_Nft', 'Received_Amount_Deposit', 'Received_Amount_Mint']),
        Received_Currency=lambda df_: combine_columns(df_, columns=[
            'Received_Currency_Buy', 'Received_Currency_Sell', 'Received_Currency_Transfer',
            'Received_Currency_Transfer_Nft', 'Received_Currency_Deposit', 'Received_Currency_Mint']))
    # Tweaking Received Currency and Sent Currency so that nfts will be accepted
    # by koinly conventions
    .assign(
        Received_Currency=lambda df_: df_['Received_Currency'].where(
            df_['Received_Currency'].isin(['ETH', 'GODS', 'IMX', np.nan]), other=df_['Received_Currency']
            .map(id_map)
            .apply(func=lambda x: 'NFT'+str(x)[:-2])),
        Sent_Currency=lambda df_: df_['Sent_Currency'].where(
            df_['Sent_Currency'].isin(['ETH', 'GODS', 'IMX', np.nan]), other=df_['Sent_Currency']
            .map(id_map)
            .apply(func=lambda x: 'NFT'+str(x)[:-2])))
    # Creating Net_Worth_Currency column
    .assign(Net_Worth_Currency = 'USD')
    # renaming columns that need no processing
    .rename(columns={'txn_id':'TxHash', 'txn_type':'Label', 'usd_amount':'Net Worth Amount'})
    # dropping unneeded columns
    .drop(columns=['txn_time', 'token_address', 'collection_name', 'collection_image_uri', 
                   'token_id', 'token_image_uri', 'Sent_Amount_Buy', 'Sent_Currency_Buy',
                   'Received_Amount_Sell', 'Received_Currency_Sell', 'Received_Amount_Transfer',
                   'Received_Currency_Transfer', 'Sent_Amount_Transfer', 'Sent_Currency_Transfer',
                   'Received_Amount_Deposit', 'Received_Currency_Deposit', 'Received_Amount_Buy',
                   'Received_Currency_Buy', 'Sent_Amount_Sell', 'Sent_Currency_Sell',
                   'Received_Amount_Mint', 'Received_Currency_Mint', 'Sent_Amount_Transfer_Nft',
                   'Sent_Currency_Transfer_Nft', 'Received_Amount_Transfer_Nft',
                   'Received_Currency_Transfer_Nft'])
    # Changing all column name underscores to spaces
    .rename(columns=lambda c: c.replace('_', ' '))
    # Changing relevant column types
    .astype({col:'category' for col in ['Label', 'token symbol', 'Net Worth Currency']})
    .astype({'Date':'datetime64'}))

In [48]:
df = tweak_dataframe(immutable)

In [49]:
df.to_csv('immutable.csv')

In [50]:
df

,TxHash,Label,from address,to address,token name,token symbol,token quantity,Net Worth Amount,Date,Sent Amount,Sent Currency,Received Amount,Received Currency,Net Worth Currency
0,124991094,mint,NaN,NaN,Lootable Corpse,NaN,NaN,NaN,2022-10-08 23:02:00,NaN,NaN,1.000000,NFT1000,USD
1,124991095,mint,NaN,NaN,On Her Command,NaN,NaN,NaN,2022-10-08 23:02:00,NaN,NaN,1.000000,NFT1001,USD
2,124991096,mint,NaN,NaN,Surpassing Blast,NaN,NaN,NaN,2022-10-08 23:02:00,NaN,NaN,1.000000,NFT1002,USD
3,124991097,mint,NaN,NaN,Engaged Healer,NaN,NaN,NaN,2022-10-08 23:02:00,NaN,NaN,1.000000,NFT1003,USD
4,124991098,mint,NaN,NaN,Ash Reader,NaN,NaN,NaN,2022-10-08 23:02:00,NaN,NaN,1.000000,NFT1004,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,184074574,transfer,0x8c8df499ad3b6f1aca2df21e70ace6dc4e0839a3,NaN,NaN,GODS,0.276447,0.083710,2023-02-05 03:31:00,NaN,NaN,0.276447,GODS,USD
294,184074575,transfer,0x8c8df499ad3b6f1aca2df21e70ace6dc4e0839a3,NaN,NaN,GODS,0.259370,0.078539,2023-02-05 03:31:00,NaN,NaN,0.259370,GODS,USD
295,185514272,transfer,0x8c8df499ad3b6f1aca2df21e70ace6dc4e0839a3,NaN,NaN,GODS,0.293501,0.079428,2023-02-10 15:59:00,NaN,NaN,0.293501,GODS,USD
296,185514274,transfer,0x8c8df499ad3b6f1aca2df21e70ace6dc4e0839a3,NaN,NaN,GODS,1.179409,0.319176,2023-02-10 15:59:00,NaN,NaN,1.179409,GODS,USD
